# Questions:
1. Which job on average pays the most?
2. Which job has the most capital?
3. Which job has the most employed people?
4. What is the difference in average salary based on each employment type?
5. Do companies tend to hire employees residing in the same country?
6. Is there a difference in pay between employees who got hired in the same year, and have the same job?

In [128]:
# pip install pandas plotly
# pip install --upgrade nbformat

In [155]:
# Importing the libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [130]:
# Importing the dataset
df = pd.read_csv("salaries.csv")

# Making the necessary changes for ease of development
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 300)

In [131]:
# Optimize the data
# 1. by keeping only the necessary columns
df = df[["work_year", "experience_level", "employment_type", "job_title", "salary_in_usd", "employee_residence", "remote_ratio", "company_location", "company_size"]]

# 2. by setting object types as categorical data
df[["experience_level", "employment_type", "job_title", "employee_residence", "company_location", "company_size"]] = df[
    ["experience_level", "employment_type", "job_title", "employee_residence", "company_location", "company_size"]
].astype("category")

In [132]:
# First 5 rows
df.head()

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,SE,FT,AI Engineer,202730,US,0,US,M
1,2024,SE,FT,AI Engineer,92118,US,0,US,M
2,2024,SE,FT,Data Engineer,130500,US,0,US,M
3,2024,SE,FT,Data Engineer,96000,US,0,US,M
4,2024,SE,FT,Machine Learning Engineer,190000,US,0,US,M


In [133]:
# Display the number of rows and columns
df.shape

(16534, 9)

In [134]:
# Display general information about the remaining columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16534 entries, 0 to 16533
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   work_year           16534 non-null  int64   
 1   experience_level    16534 non-null  category
 2   employment_type     16534 non-null  category
 3   job_title           16534 non-null  category
 4   salary_in_usd       16534 non-null  int64   
 5   employee_residence  16534 non-null  category
 6   remote_ratio        16534 non-null  int64   
 7   company_location    16534 non-null  category
 8   company_size        16534 non-null  category
dtypes: category(6), int64(3)
memory usage: 511.9 KB


In [135]:
# Check for nulls
df.isna().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [136]:
# Check for duplicates
df.duplicated().sum()

6422

In [137]:
# Check the data types of the columns
df.dtypes

work_year                int64
experience_level      category
employment_type       category
job_title             category
salary_in_usd            int64
employee_residence    category
remote_ratio             int64
company_location      category
company_size          category
dtype: object

In [138]:
# Answer to "1. Which job on average pays the most"

# Step 1: Calculate the employee count for each job title
employee_count = df["job_title"].value_counts()

# Step 2: Filter the job titles to include only those with more than 20 employees
filtered_jobs = employee_count[employee_count > 10].index

# Step 3: Calculate the average salary for these filtered job titles
highest_paying_job = df[df["job_title"].isin(filtered_jobs)].groupby("job_title")["salary_in_usd"].mean().sort_values(ascending=False).reset_index()

# Step 4: Select the top 5 highest paying jobs
top_5_jobs = highest_paying_job.iloc[:5]

# Step 5: Plot the graph using the filtered data
fig = px.bar(top_5_jobs, x="job_title", y="salary_in_usd", title=f"The top 5 highest paying jobs in Data Science are: {top_5_jobs['job_title'].tolist()}", labels={"job_title": "Job Title", "salary_in_usd": "Salary in USD"})

fig.show()

In [139]:
# Answer to "2. Which job has the most capital?"

# Step 1: Calculate the employee count for each job title
employee_count = df["job_title"].value_counts()

# Step 2: Filter the job titles to include only those with more than 20 employees
filtered_jobs = employee_count[employee_count > 10].index

# Step 3: Calculate the total salary for these filtered job titles
highest_paying_job = df[df["job_title"].isin(filtered_jobs)].groupby("job_title")["salary_in_usd"].sum().sort_values(ascending=False).reset_index()

# Step 4: Select the top 5 highest capital jobs
top_5_jobs = highest_paying_job.iloc[:5]

# Step 5: Plot the graph using the filtered data
fig = px.bar(top_5_jobs, x="job_title", y="salary_in_usd", title=f"The top 5 highest capital jobs in Data Science are: {top_5_jobs['job_title'].tolist()}", labels={"job_title": "Job Title", "salary_in_usd": "Salary in USD"})

fig.show()

In [157]:
# Answer to "3. Which job has the most employed people?"

# Step 1: Calculate the employee count for each job title
employee_count = df["job_title"].value_counts()

# Step 2: Filter the job titles to include only those with more than 20 employees
filtered_jobs = employee_count[employee_count > 20].index

# Step 3: Calculate the average salary for these filtered job titles
highest_paying_job = df[df["job_title"].isin(filtered_jobs)].groupby("job_title")["salary_in_usd"].mean().sort_values(ascending=False).reset_index()

# Step 4: Select the top 7 highest paying jobs
top_7_jobs = highest_paying_job.head(7)["job_title"]

# Step 5: Calculate the average salary per year for each of the top 7 job titles
df_top_7 = df[df["job_title"].isin(top_7_jobs)]
average_salary_per_year = df_top_7.groupby(["job_title", "work_year"])["salary_in_usd"].mean().reset_index()

# Step 6: Create the plot with traces for each job title
fig = go.Figure()

for job in top_7_jobs:
    job_data = average_salary_per_year[average_salary_per_year["job_title"] == job]
    fig.add_trace(go.Scatter(
        x=job_data["work_year"],
        y=job_data["salary_in_usd"],
        mode='lines+markers',
        name=job
    ))

# Customize the layout
fig.update_layout(
    title="Average Salary Per Year for Top 7 Highest Paying Jobs in Data Science",
    xaxis_title="Year",
    yaxis_title="Average Salary in USD",
    legend_title="Job Title"
)

# Show the plot
fig.show()

# This shows a potential saturation in the market since the trend for data science started.
# The salaries have been higher than expected in the first year.
# Employers saw there is no reason to increase the salaries since the demand is high.
# The salaries have been more stable since then.